## Trabajo en clase
### Alexa Amaguaya,Glenn Salcedo, Kleber Janeta

In [53]:
import pandas as pd
import re

Cargamos la base de datos


In [120]:
# Reemplaza 'ruta_del_archivo.csv' con la ruta real de tu archivo CSV
ruta_archivo = '/content/wiki_movie_plots_deduped.csv'

# Leer el CSV y cargarlo en un DataFrame
df = pd.read_csv(ruta_archivo,usecols=['Title','Plot'])
df=df.head(10)

# Mostrar los primeros registros del DataFrame
print(df.head())


                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  Lasting just 61 seconds and consisting of two ...  
4  The earliest known adaptation of the classic f...  


In [121]:
df['Indice'] = range(1, len(df) + 1)

In [122]:
def limpiezaTexto(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    text = re.sub(r'[^a-zA-Z\s]+', ' ', text)
    # eliminar palabras vacias
    text= text.strip()
    return text

In [123]:
df['textoLimpio'] = df['Plot'].apply(limpiezaTexto)
df

,Title,Plot,Indice,textoLimpio
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",1,a bartender is working at a saloon serving dr...
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",2,the moon painted with a smiling face hangs ov...
2,The Martyred Presidents,"The film, just over a minute long, is composed...",3,the film just over a minute long is composed...
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,4,lasting just seconds and consisting of two s...
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...,5,the earliest known adaptation of the classic f...
5,Alice in Wonderland,"Alice follows a large white rabbit down a ""Rab...",6,alice follows a large white rabbit down a rab...
6,The Great Train Robbery,The film opens with two bandits breaking into ...,7,the film opens with two bandits breaking into ...
7,The Suburbanite,The film is about a family who move to the sub...,8,the film is about a family who move to the sub...
8,The Little Train Robbery,The opening scene shows the interior of the ro...,9,the opening scene shows the interior of the ro...
9,The Night Before Christmas,Scenes are introduced using lines of the poem....,10,scenes are introduced using lines of the poem ...


In [78]:
pip install  nltk # intalaicon de nltk

In [79]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # descarga de stopwords en ingles
stop_words = set(stopwords.words('english'))  # obtencion de stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
# Función para eliminar las stopwords
def eliminar_stopwords(texto):
    if not isinstance(texto, str):  # Manejar valores vacíos o nulos
        return texto
    palabras = texto.split() #tokenizacion de palabras
    palabras_sin_stopwords = [palabra for palabra in palabras if palabra.lower() not in stop_words] # for para elimianr stpwords y colocar el texto sin stopwords en una nueva lista
    return palabras_sin_stopwords # transformacion de tokens a texto

In [125]:
# Aplicar la función a la columna
df['sinstopwords'] = df['textoLimpio'].apply(eliminar_stopwords)

In [126]:
from collections import defaultdict

def crear_indice_invertido_df(df, col_id, col_texto):
    indice_invertido = defaultdict(list)

    for _, fila in df.iterrows():
        doc_id = fila[col_id]
        texto = fila[col_texto]

        # Asume que 'texto' ya está tokenizado y es una lista de términos.
        for palabra in texto:
            if doc_id not in indice_invertido[palabra]:
                indice_invertido[palabra].append(doc_id)

    return dict(indice_invertido)



In [127]:
def buscar_titulos_por_palabra(df, indice, palabra, col_id, col_titulo):
    """
    Busca una palabra en el índice invertido y devuelve los títulos de las películas asociadas.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene los documentos.
    - indice (dict): Índice invertido.
    - palabra (str): La palabra que deseas buscar.
    - col_id (str): Nombre de la columna que contiene los IDs de los documentos.
    - col_titulo (str): Nombre de la columna que contiene los títulos.

    Retorna:
    - list: Lista de títulos de películas que contienen la palabra.
    """
    doc_ids = indice.get(palabra, [])
    titulos = df[df[col_id].isin(doc_ids)][col_titulo].tolist()
    return titulos

In [128]:
# Crear el índice invertido
indice = crear_indice_invertido_df(df, col_id='Indice', col_texto='sinstopwords')

# Buscar una palabra específica en el índice invertido
palabra = 'man'  # Cambia esta palabra por la que quieras buscar
resultado = indice.get( palabra,indice)
titulos = buscar_titulos_por_palabra(df, indice, palabra, col_id='Indice', col_titulo='Title')
# Mostrar el resultado
print(f"Documentos que contienen la palabra '{palabra}': {resultado,titulos}")

Documentos que contienen la palabra 'man': ([1, 2], ['Kansas Saloon Smashers', 'Love by the Light of the Moon'])


2. WOSH

In [129]:
pip install whoosh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 26.3 MB/s eta 0:00:00


In [130]:
df

,Title,Plot,Indice,textoLimpio,sinstopwords
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",1,a bartender is working at a saloon serving dr...,"[bartender, working, saloon, serving, drinks, ..."
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",2,the moon painted with a smiling face hangs ov...,"[moon, painted, smiling, face, hangs, park, ni..."
2,The Martyred Presidents,"The film, just over a minute long, is composed...",3,the film just over a minute long is composed...,"[film, minute, long, composed, two, shots, fir..."
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,4,lasting just seconds and consisting of two s...,"[lasting, seconds, consisting, two, shots, fir..."
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...,5,the earliest known adaptation of the classic f...,"[earliest, known, adaptation, classic, fairyta..."
5,Alice in Wonderland,"Alice follows a large white rabbit down a ""Rab...",6,alice follows a large white rabbit down a rab...,"[alice, follows, large, white, rabbit, rabbit,..."
6,The Great Train Robbery,The film opens with two bandits breaking into ...,7,the film opens with two bandits breaking into ...,"[film, opens, two, bandits, breaking, railroad..."
7,The Suburbanite,The film is about a family who move to the sub...,8,the film is about a family who move to the sub...,"[film, family, move, suburbs, hoping, quiet, l..."
8,The Little Train Robbery,The opening scene shows the interior of the ro...,9,the opening scene shows the interior of the ro...,"[opening, scene, shows, interior, robbers, den..."
9,The Night Before Christmas,Scenes are introduced using lines of the poem....,10,scenes are introduced using lines of the poem ...,"[scenes, introduced, using, lines, poem, santa..."


In [137]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, NUMERIC
from whoosh.qparser import QueryParser
import os

def crear_indice(data, ruta_indice):
    """
    Crea un índice en Whoosh para los datos proporcionados.

    Parámetros:
    - data (list of dict): Lista de diccionarios con los datos a indexar.
    - ruta_indice (str): Carpeta donde se almacenará el índice.

    Retorna:
    - None
    """
    if not os.path.exists(ruta_indice):
        os.mkdir(ruta_indice)

    # Definir el esquema del índice
    schema = Schema(title=TEXT(stored=True), release_year=NUMERIC(stored=True), content=TEXT)

    # Crear el índice
    ix = create_in(ruta_indice, schema)
    writer = ix.writer()

    # Agregar documentos al índice
    for doc in data:
        writer.add_document(
            Title=doc["Title"],
            Plot=doc["Plot"],
        )
    writer.commit()

def buscar_en_indice(consulta, ruta_indice):
    """
    Realiza una búsqueda en el índice de Whoosh.

    Parámetros:
    - consulta (str): La consulta a realizar.
    - ruta_indice (str): Carpeta donde se encuentra el índice.

    Retorna:
    - list of dict: Documentos que coinciden con la búsqueda.
    """
    from whoosh.index import open_dir

    # Abrir el índice
    ix = open_dir(ruta_indice)
    with ix.searcher() as searcher:
        query = QueryParser("Plot", ix.schema).parse(consulta)
        resultados = searcher.search(query)
        return [{"Title": r["Title"], "Plot": r["Plot"]} for r in resultados]


# Crear el índice
ruta_indice = "/content/indice_whoosh"
crear_indice(df, ruta_indice)

# Realizar una búsqueda
consulta = "man"
resultados = buscar_en_indice(consulta, ruta_indice)

# Mostrar los resultados
print(f"Resultados para la consulta '{consulta}':")
for res in resultados:
    print(f"Título: {res['title']}, Año de lanzamiento: {res['release_year']}")


LockError: 